In [ ]:
import gymnasium as gym
from model import DQN
from torch import optim
from utils import Transition, ReplayMemory, VideoRecorder
from wrapper import AtariWrapper
import random
import torch
import torch.nn as nn
from itertools import count
import os
import matplotlib.pyplot as plt
import math
from collections import deque
import ale_py
from torch.utils.tensorboard import SummaryWriter

gym.register_envs(ale_py)

device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer

# some hyperparameters
GAMMA = 0.99  # bellman function
EPS_START = 1.0  # Start with full exploration
EPS_END = 0.05  # Minimum exploration rate
EPS_DECAY = 50000  # Slower decay
WARMUP = 5000  # don't update net until WARMUP steps
TARGET_UPDATE = 10000
LR = 1e-5
BATCH_SIZE = 128
N_EPISODES = 2000
EVAL_CYCLE = N_EPISODES / 10

steps_done = 0
eps_threshold = EPS_START


def select_action(state: torch.Tensor) -> torch.Tensor:
    global eps_threshold
    global steps_done

    # Exponential decay of exploration rate
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(
        -1.0 * steps_done / EPS_DECAY
    )

    # Ensure eps_threshold doesn't get stuck
    eps_threshold = max(eps_threshold, EPS_END)

    sample = random.random()
    steps_done += 1

    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]]).to(device)


env = gym.make("SpaceInvadersNoFrameskip-v4")
env = AtariWrapper(env)

# n_action = 6
n_action = env.action_space.n

log_dir = os.path.join("logs")
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
log_path = os.path.join(log_dir, "log.txt")

# tensorboard
writer = SummaryWriter(log_dir=log_dir)


# video
video = VideoRecorder(log_dir)

# create network and target network
policy_net = DQN(in_channels=4, n_actions=n_action).to(device)
target_net = DQN(in_channels=4, n_actions=n_action).to(device)

# let target model = model
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

# replay memory
memory = ReplayMemory(50000)

# optimizer
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)

# warming up
print("Warming up...")
warmupstep = 0
for _ in range(WARMUP):
    obs, info = env.reset()  # (84,84)
    obs = torch.from_numpy(obs).to(device)  # (84,84)
    # stack four frames together, hoping to learn temporal info
    obs = torch.stack((obs, obs, obs, obs)).unsqueeze(0)  # (1,4,84,84)
    done = False

    # step loop
    while not done:
        warmupstep += 1
        # take one step
        action = torch.tensor([[env.action_space.sample()]]).to(device)
        next_obs, reward, terminated, truncated, info = env.step(action.item())
        done = terminated or truncated

        # convert to tensor
        reward = torch.tensor([reward], device=device)  # (1)
        done = torch.tensor([done], device=device)  # (1)
        next_obs = torch.from_numpy(next_obs).to(device)  # (84,84)
        next_obs = torch.stack((next_obs, obs[0][0], obs[0][1], obs[0][2])).unsqueeze(
            0
        )  # (1,4,84,84)

        # store the transition in memory
        memory.push(obs, action, next_obs, reward, done)

        # move to next state
        obs = next_obs

rewardList = []
lossList = []
rewarddeq = deque([], maxlen=100)
lossdeq = deque([], maxlen=100)
avgrewardlist = []
avglosslist = []


def eval_model():
    video.reset()
    evalenv = gym.make("SpaceInvadersNoFrameskip-v4")
    evalenv = AtariWrapper(evalenv, video=video)
    obs, info = evalenv.reset()
    obs = torch.from_numpy(obs).to(device)
    obs = torch.stack((obs, obs, obs, obs)).unsqueeze(0)
    evalreward = 0
    policy_net.eval()

    for _ in count():
        action = policy_net(obs).max(1)[1]
        next_obs, reward, terminated, truncated, info = evalenv.step(action.item())
        evalreward += reward
        next_obs = torch.from_numpy(next_obs).to(device)  # (84,84)
        next_obs = torch.stack((next_obs, obs[0][0], obs[0][1], obs[0][2])).unsqueeze(
            0
        )  # (1,4,84,84)
        obs = next_obs
        if terminated or truncated:
            if info["lives"] == 0:  # real end
                break
            else:
                obs, info = evalenv.reset()
                obs = torch.from_numpy(obs).to(device)
                obs = torch.stack((obs, obs, obs, obs)).unsqueeze(0)

    evalenv.close()
    video.save(f"{epoch}.mp4")
    torch.save(policy_net, os.path.join(log_dir, f"model{epoch}.pth"))
    print(f"Eval epoch {epoch}: Reward {evalreward}")


# epoch loop
for epoch in range(N_EPISODES + 1):
    obs, info = env.reset()  # (84,84)
    obs = torch.from_numpy(obs).to(device)  # (84,84)
    # stack four frames together, hoping to learn temporal info
    obs = torch.stack((obs, obs, obs, obs)).unsqueeze(0)  # (1,4,84,84)

    total_loss = 0.0
    total_reward = 0

    # step loop
    for step in count():
        # take one step
        action = select_action(obs)
        next_obs, reward, terminated, truncated, info = env.step(action.item())
        total_reward += reward
        done = terminated or truncated

        # convert to tensor
        reward = torch.tensor([reward], device=device)  # (1)
        done = torch.tensor([done], device=device)  # (1)
        next_obs = torch.from_numpy(next_obs).to(device)  # (84,84)
        next_obs = torch.stack((next_obs, obs[0][0], obs[0][1], obs[0][2])).unsqueeze(
            0
        )  # (1,4,84,84)

        # store the transition in memory
        memory.push(obs, action, next_obs, reward, done)

        # move to next state
        obs = next_obs

        # train
        policy_net.train()
        transitions = memory.sample(BATCH_SIZE)
        batch = Transition(
            *zip(*transitions)
        )  # batch-array of Transitions -> Transition of batch-arrays.
        state_batch = torch.cat(batch.state)  # (bs,4,84,84)
        next_state_batch = torch.cat(batch.next_state)  # (bs,4,84,84)
        action_batch = torch.cat(batch.action)  # (bs,1)
        reward_batch = torch.cat(batch.reward).unsqueeze(1)  # (bs,1)
        done_batch = torch.cat(batch.done).unsqueeze(1)  # (bs,1)

        # Q(st,a)
        state_qvalues = policy_net(state_batch)  # (bs,n_actions)
        selected_state_qvalue = state_qvalues.gather(1, action_batch)  # (bs,1)

        with torch.no_grad():
            # Q'(st+1,a)
            next_state_target_qvalues = target_net(next_state_batch)  # (bs,n_actions)
            next_state_selected_qvalue = next_state_target_qvalues.max(1, keepdim=True)[
                0
            ]  # (bs,1)

        # td target
        tdtarget = (
            next_state_selected_qvalue * GAMMA * ~done_batch + reward_batch
        )  # (bs,1)

        # optimize
        criterion = nn.SmoothL1Loss()
        loss = criterion(selected_state_qvalue, tdtarget)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # let target_net = policy_net every 1000 steps
        if steps_done % TARGET_UPDATE == 0:
            target_net.load_state_dict(policy_net.state_dict())

        if done:
            # eval
            if epoch % EVAL_CYCLE == 0:
                with torch.no_grad():
                    eval_model()
            break

    rewardList.append(total_reward)
    lossList.append(total_loss)
    rewarddeq.append(total_reward)
    lossdeq.append(total_loss)
    avgreward = sum(rewarddeq) / len(rewarddeq)
    avgloss = sum(lossdeq) / len(lossdeq)
    avglosslist.append(avgloss)
    avgrewardlist.append(avgreward)

    writer.add_scalar("Reward/Total", total_reward, epoch)
    writer.add_scalar("Loss/Total", total_loss, epoch)
    writer.add_scalar("Metrics/Average Reward", avgreward, epoch)
    writer.add_scalar("Metrics/Average Loss", avgloss, epoch)
    writer.add_scalar("Hyperparameters/Epsilon", eps_threshold, epoch)
    writer.add_scalar("Metrics/Steps Done", steps_done, epoch)

    output = f"Epoch {epoch}: Loss {total_loss:.2f}, Reward {total_reward}, Avgloss {avgloss:.2f}, Avgreward {avgreward:.2f}, Epsilon {eps_threshold:.2f}, TotalStep {steps_done}"
    print(output)
    with open(log_path, "a") as f:
        f.write(f"{output}\n")

env.close()


# plot loss-epoch and reward-epoch
plt.figure(1)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(range(len(lossList)), lossList, label="loss")
plt.plot(range(len(lossList)), avglosslist, label="avg")
plt.legend()
plt.savefig(os.path.join(log_dir, "loss.png"))

plt.figure(2)
plt.xlabel("Epoch")
plt.ylabel("Reward")
plt.plot(range(len(rewardList)), rewardList, label="reward")
plt.plot(range(len(rewardList)), avgrewardlist, label="avg")
plt.legend()
plt.savefig(os.path.join(log_dir, "reward.png"))

6


SyntaxError: 'return' outside function (3135209932.py, line 77)